Version 1.1.0

# Mean encodings

In this programming assignment you will be working with `1C` dataset from the final competition. You are asked to encode `item_id` in 4 different ways:

    1) Via KFold scheme;  
    2) Via Leave-one-out scheme;
    3) Via smoothing scheme;
    4) Via expanding mean scheme.

**You will need to submit** the correlation coefficient between resulting encoding and target variable up to 4 decimal places.

### General tips

* Fill NANs in the encoding with `0.3343`.
* Some encoding schemes depend on sorting order, so in order to avoid confusion, please use the following code snippet to construct the data frame. This snippet also implements mean encoding without regularization.

In [2]:
import pandas as pd
import numpy as np
from itertools import product
from grader import Grader

# Read data

In [3]:
sales = pd.read_csv('../readonly/final_project_data/sales_train.csv.gz')

# Aggregate data

Since the competition task is to make a monthly prediction, we need to aggregate the data to montly level before doing any encodings. The following code-cell serves just that purpose.

In [4]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})

#fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [5]:
gb.tail(5)

,shop_id,item_id,date_block_num,target
1609119,59,22164,27,2.0
1609120,59,22164,30,1.0
1609121,59,22167,9,1.0
1609122,59,22167,11,2.0
1609123,59,22167,17,1.0


In [6]:
sales.tail(5)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2935844,10.10.2015,33,25,7409,299.0,1.0
2935845,09.10.2015,33,25,7460,299.0,1.0
2935846,14.10.2015,33,25,7459,349.0,1.0
2935847,22.10.2015,33,25,7440,299.0,1.0
2935848,03.10.2015,33,25,7460,299.0,1.0


In [23]:
all_data.shape

(10913850, 5)

# Mean encodings without regularization

After we did the techinical work, we are ready to actually *mean encode* the desired `item_id` variable. 

Here are two ways to implement mean encoding features *without* any regularization. You can use this code as a starting point to implement regularized techniques. 

#### Method 1

In [8]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()
print(item_id_target_mean)
# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation between target values and predicted values.
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

item_id
0        0.020000
1        0.023810
2        0.019802
3        0.019802
4        0.020000
5        0.020000
6        0.020000
7        0.020000
8        0.019802
9        0.019608
10       0.020000
11       0.020000
12       0.021739
13       0.020000
14       0.020000
15       0.020000
16       0.020000
17       0.020000
18       0.019608
19       0.022222
20       0.019608
21       0.020000
22       0.021277
23       0.021277
24       0.021277
25       0.019608
26       0.019231
27       0.056834
28       0.141176
29       0.037383
           ...   
22140    0.191461
22141    0.276042
22142    0.069930
22143    1.930804
22144    0.266667
22145    0.650407
22146    0.040334
22147    0.082100
22148    0.021739
22149    0.058923
22150    0.088821
22151    1.159375
22152    0.155109
22153    0.026627
22154    0.109870
22155    0.093671
22156    0.029197
22157    0.021978
22158    0.022727
22159    0.172414
22160    0.097030
22161    0.022222
22162    1.556793
22163    0.581395
22

#### Method 2

In [9]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.483038698862


See the printed value? It is the correlation coefficient between the target variable and your new encoded feature. You need to **compute correlation coefficient** between the encodings, that you will implement and **submit those to coursera**.

In [10]:
grader = Grader()

# 1. KFold scheme

Explained starting at 41 sec of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

**Now it's your turn to write the code!** 

You may use 'Regularization' video as a reference for all further tasks.

First, implement KFold scheme with five folds. Use KFold(5) from sklearn.model_selection. 

1. Split your data in 5 folds with `sklearn.model_selection.KFold` with `shuffle=False` argument.
2. Iterate through folds: use all but the current fold to calculate mean target for each level `item_id`, and  fill the current fold.

    *  See the **Method 1** from the example implementation. In particular learn what `map` and pd.Series.map functions do. They are pretty handy in many situations.

In [11]:
# YOUR CODE GOES HERE\

from sklearn.model_selection import KFold
skf = KFold(shuffle=False, n_splits = 5)

for train_index, val_index in skf.split(all_data):
    x_tr = all_data.iloc[train_index]
    #get mean of target of each item_id in training set
    mean = x_tr.groupby("item_id").target.mean()
    print(mean.shape)
    # map = outer join. join 2 series(columns) by mapping the id.
    all_data.loc[all_data.index[val_index],'item_target_enc'] = all_data.loc[all_data.index[val_index],'item_id'].map(mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 
# You will need to compute correlation like that
encoded_feature = all_data['item_target_enc'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('KFold_scheme', corr)

(20205,)
(21289,)
(21521,)
(21416,)
(19244,)
0.41645907128
Current answer for task KFold_scheme is: 0.41645907128


In [12]:
all_data.loc[all_data.index[val_index],'item_id'].map(mean).shape

(2182770,)

# 2. Leave-one-out scheme

Now, implement leave-one-out scheme. Note that if you just simply set the number of folds to the number of samples and run the code from the **KFold scheme**, you will probably wait for a very long time. 

To implement a faster version, note, that to calculate mean target value using all the objects but one *given object*, you can:

1. Calculate sum of the target values using all the objects.
2. Then subtract the target of the *given object* and divide the resulting value by `n_objects - 1`. 

Note that you do not need to perform `1.` for every object. And `2.` can be implemented without any `for` loop.

It is the most convenient to use `.transform` function as in **Method 2**.

In [13]:
# YOUR CODE GOES HERE
totalSum = all_data['item_id'].map(all_data.groupby('item_id')['target'].sum())
n_objects = all_data['item_id'].map(all_data.groupby('item_id')['target'].count())
all_data['item_target_enc'] = (totalSum - all_data['target']) / (n_objects - 1)
encoded_feature = all_data['item_target_enc'].values

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Leave-one-out_scheme', corr)


0.480384831129
Current answer for task Leave-one-out_scheme is: 0.480384831129


# 3. Smoothing

Explained starting at 4:03 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Next, implement smoothing scheme with $\alpha = 100$. Use the formula from the first slide in the video and $0.3343$ as `globalmean`. Note that `nrows` is the number of objects that belong to a certain category (not the number of rows in the dataset).

In [18]:
# YOUR CODE GOES HERE
alpha = 100
globalmean = 0.3343
nrows = all_data.groupby('item_id')['target'].count()
targetmean = all_data.groupby('item_id')['target'].mean()
smooth = (targetmean*nrows + globalmean*alpha) / (nrows + alpha)
all_data['item_target_enc'] = all_data['item_id'].map(smooth)
encoded_feature = all_data['item_target_enc'].values

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Smoothing_scheme', corr)

0.48181987971
Current answer for task Smoothing_scheme is: 0.48181987971


# 4. Expanding mean scheme

Explained starting at 5:50 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Finally, implement the *expanding mean* scheme. It is basically already implemented for you in the video, but you can challenge yourself and try to implement it yourself. You will need [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) and [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html) functions from pandas.

In [22]:
# YOUR CODE GOES HERE
cumsum = all_data.groupby('item_id')['target'].cumsum() - all_data['target']
cumcnt = all_data.groupby('item_id').cumcount()

print(cumsum)
print(cumcnt)
all_data['item_target_enc'] = cumsum/cumcnt
all_data['item_target_enc'].fillna(0.3343, inplace=True)
encoded_feature = all_data['item_target_enc'].values

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Expanding_mean_scheme', corr)

139255         0.0
141495         0.0
144968         0.0
142661         0.0
138947         0.0
138948         0.0
138949         0.0
139247         0.0
142672         0.0
142065         0.0
139208         0.0
142670         0.0
139207         0.0
138950         0.0
143764         0.0
141505         0.0
139199         0.0
138952         0.0
139176         0.0
138951         0.0
139177         0.0
139178         0.0
139179         0.0
143769         0.0
142671         0.0
144539         0.0
139180         0.0
138953         0.0
144265         0.0
141744         0.0
             ...  
10772600    1209.0
10770510     126.0
10769953     108.0
10769955    1664.0
10768833     184.0
10769961     420.0
10770625     143.0
10769956     695.0
10771598    2807.0
10767854    3441.0
10768086    5271.0
10768087    1191.0
10768088    2089.0
10767847     389.0
10769954     260.0
10767848    1707.0
10767849     310.0
10768089     104.0
10770726     131.0
10772844     187.0
10771530     121.0
10768090    

## Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. Note: Token expires 30 minutes after generation.

In [20]:
STUDENT_EMAIL = "klau@edu.hse.ru"
STUDENT_TOKEN = "7oXJL8vz9UvwrmU9"
grader.status()

You want to submit these numbers:
Task KFold_scheme: 0.41645907128
Task Leave-one-out_scheme: 0.480384831129
Task Smoothing_scheme: 0.48181987971
Task Expanding_mean_scheme: 0.502524521108


In [21]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

Submitted to Coursera platform. See results on assignment page!
